In [1]:
%pip install langchain-community langchain-core
%pip install sklearn
%pip install pandas
%pip install nltk


You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import rdflib
import random
from typing import List
from langchain_community.graphs import OntotextGraphDBGraph
from constants import (
    TRAINED_MODEL_ID_PATH,
    TEST_DATA_PATH,
    SPARQL_FIX_PROMPT,
    QA_PROMPT,
    SPARQL_GENERATION_PROMPT,
)
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm

In [3]:
class CustomOntotextGraphDBGraph(OntotextGraphDBGraph):
    def query(
        self,
        query: str,
    ) -> List[rdflib.query.ResultRow]:
        """
        Query the graph.
        """
        from rdflib.query import ResultRow

        res = self.graph.query(query)
        if res.type == "ASK":
            return [r for r in res if isinstance(r, bool)]
        return [r for r in res if isinstance(r, ResultRow)]

In [4]:
config = {
    "query_endpoint": "http://localhost:7200/repositories/imkg",
    "local_file": "/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/KG/iMKG.ttl",
    "local_file_format": "turtle",
}
try:
    graph = CustomOntotextGraphDBGraph(**config)
    print("Graph object created successfully.")
except Exception as e:
    print("Failed to create graph object:", e)

Graph object created successfully.


In [5]:
# Define the path to the test data
TEST_DATA_PATH = os.path.join(os.getcwd(), "data", "qa_test.json")
TEST_DATA_RESULTS_PATH = os.path.join(os.getcwd(), "data", "qa_test_results.json")

# Load the dataset
with open(TEST_DATA_PATH, "r", encoding="utf-8") as f:
    test_dataset = json.load(f)

# Select 10 random test data points
test_data_points = random.sample(test_dataset, 10)

In [12]:
import requests


def get_generated_queries(questions: list):
    response = requests.post(
        "http://localhost:8001/query",
        json={"questions": questions},  # Send batch request
    )
    response_json = response.json()
    return {
        item["question"]: item["response"] for item in response_json
    }  # Convert to a dictionary


def compare_two_query(correct_query: str, generated_query: str):
    generated_results = set(graph.query(generated_query))
    sample_results = set(graph.query(correct_query))
    return generated_results == sample_results


batch_size = 1
for i in range(0, len(test_data_points), batch_size):
    batch = test_data_points[i : i + batch_size]

    # Extract questions for the batch
    questions = [data_point["question"] for data_point in batch]

    # Get batch query responses as a dictionary
    generated_query_map = get_generated_queries(questions)

    # Process results for the batch
    for index, data_point in enumerate(batch, start=i + 1):
        question = data_point["question"]
        generated_query = generated_query_map.get(question)

        if not generated_query:
            print(f"Generated query is invalid for question: {question}")
            continue

        data_point["generated_query"] = generated_query
        data_point["is_generated_query_correct"] = compare_two_query(
            correct_query=data_point["sparql"], generated_query=generated_query
        )
        print(f"Question: {index} / {len(test_data_points)}\n{data_point}")

Question: 1 / 10
{'qid': 'mcqa-928ec523-e67e-11ee-94a4-58961d663d9c', 'question': 'which films have the same director as [Sunshine State] and are of the genre [drama]?', 'question_type': 'movie_to_director_to_movie_constraint_genre', 'topic_entity': ['Sunshine State', 'drama'], 'topic_entity_id': ['https://www.wikidata.org/entity/Q1670513', 'https://www.wikidata.org/entity/Q130232', 'https://www.wikidata.org/entity/Q21010853', 'https://www.wikidata.org/entity/Q25372', 'https://www.wikidata.org/entity/MT95'], 'answer': ['Casa de los Babys', 'Honeydripper', 'Go for Sisters', 'Lianna', 'Return of the Secaucus 7', 'City of Hope', 'Eight Men Out', 'Men with Guns', 'Matewan', 'Passion Fish', 'Amigo', 'Limbo', 'Silver City', 'Lone Star'], 'answer_id': ['https://www.wikidata.org/entity/Q1046576', 'https://www.wikidata.org/entity/Q1626868', 'https://www.wikidata.org/entity/Q16948216', 'https://www.wikidata.org/entity/Q1961839', 'https://www.wikidata.org/entity/Q2195648', 'https://www.wikidata.o

In [14]:
with open(TEST_DATA_RESULTS_PATH, "w", encoding="utf-8") as f:
    json.dump(test_data_points, f)